## Polygon Exploratory Data Analysis
Exploratory Data Analysis with 10-years of Stock and ETF data from the US.

This notebook targets the basic strategy concepts of algorithmic trading, namely:
- Mean reversion
- Trend following/Momentum trading
- Statistical Arbitrage

To that end, the following analyses will be conducted:
- Moving average (simple and exponential)
- Volatility measures (ATR/Standard Deviation) across equities and time
- Age of equities
- Correlation between equities

2D-plots can be plotted as heat maps

The effect of significant dates/events can also be studied (eg holidays, ex-dividends, splits)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import os, glob

dir_name = "data/10year_1hourcandle_us_etf_stock/data/"

# From API documentation
column_def = {
    "o": "open_price",
    "c": "close_price",
    "h": "high_price",
    "l": "low_price",
    "n": "num_transactions", # number of transactions in window
    "t": "timestamp", # Unix Milliseconds
    "v": "volume",
    "vw": "vol_weighted_average_price"
}

etf_dfs = []

for filename in glob.glob(os.path.join(dir_name, "*.csv")):
    if os.path.isfile:
        df = pd.read_csv(filename, header=0, delimiter=',', index_col=0)

        ticker = filename.split('\\')[-1][:-4]
        df['ticker'] = ticker
        
        df.rename(column_def, inplace=True, axis=1)

        etf_dfs.append(df)
    

agg_df = pd.concat(etf_dfs, axis=0, join='outer', ignore_index=True)

print(agg_df.shape[0])
agg_df.head()

In [ ]:
agg_df.isnull().sum()

We're not too concerned about the volume weighted average price or the number of transactions, so we move on.

It will be easier to deal with the individual DataFrames, so we will use the `etf_dfs` list of DataFrames for analysis.

### Moving Averages

First calculate the moving averages. Each entry is assumed to be a day.

In [ ]:
# Simple moving average
def sma(df):
    periods = (5, 10, 20, 50, 100, 200)
    for period in periods:
        df[f'{period:.0f}_day_sma'] = df['close_price'].rolling(period).mean()
    return df

for i in range(len(etf_dfs)):
    etf_dfs[i] = sma(etf_dfs[i])

etf_dfs[0].head(20)

In [ ]:
# Exponential moving average
def ema(df):
    smoothing = 2

    periods = (12, 26, 50, 200)
    close = df['close_price'].to_numpy() # Convert to numpy array

    for period in periods:
        # continue if the period exceeds the length of the data
        if period > df.shape[0]:
            continue

        ema = np.empty(df.shape[0])

        # The first <period> days should be empty
        for i in range(period):
            ema[i] = np.NaN

        # Calculate <period>+1 EMA using the SMA of the first <period> days
        initial_sma = np.mean(close[:period])
        ema[period] = (close[period] * (smoothing / (1+period))) + (initial_sma * (1 - (smoothing / (1+period))))

        # Calculate the rest of the EMAs
        for i in range(period+1, len(close)):
            ema[i] = (close[i] * (smoothing / (1+period))) + (ema[i-1] * (1 - (smoothing / (1+period))))
        df[f'{period:.0f}_day_ema'] = pd.Series(ema)

    return df

for i in range(len(etf_dfs)):
    etf_dfs[i] = ema(etf_dfs[i])

etf_dfs[0].head(20)

In [ ]:
import datetime

# Visualisation
fig, axs = plt.subplots(4, 2)

for i in range(len(etf_dfs)):
    try:
        ema_12 = etf_dfs[i]["12_day_ema"]
        ema_200 = etf_dfs[i]["200_day_ema"]
        ts = etf_dfs[i]["timestamp"].map(lambda x: datetime.datetime.fromtimestamp(x/1000))
    except:
        # Less than 200 days old (or less than 12 days old, but unlikely)
        continue

    row = i % 4
    column = i // 4
    axs[row, column].set(ylabel="EMA", xlabel="Time")
    axs[row, column].set_title(etf_dfs[i].loc[0, "ticker"])
    axs[row, column].plot(ts, ema_12, label="12 day")
    axs[row, column].plot(ts, ema_200, label="200 day")

fig.tight_layout()
plt.show()

### Volatility

#### Standard Deviation
Calculate the Standard Deviation. Each entry is assumed to be a day.

In [ ]:
def stddev(df, period=14):
    df[f'{period:.0f}_day_stddev'] = df['close_price'].rolling(period).std()
    return df

for i in range(len(etf_dfs)):
    etf_dfs[i] = stddev(etf_dfs[i])

etf_dfs[0].head(20)

In [ ]:
import datetime

# Visualisation
fig, axs = plt.subplots(4, 2)

for i in range(len(etf_dfs)):
    try:
        stddev = etf_dfs[i]["14_day_stddev"]
        ts = etf_dfs[i]["timestamp"].map(lambda x: datetime.datetime.fromtimestamp(x/1000))
    except:
        # Less than 14 days old
        continue

    row = i % 4
    column = i // 4
    axs[row, column].set(ylabel="EMA", xlabel="Time")
    axs[row, column].set_title(etf_dfs[i].loc[0, "ticker"])
    axs[row, column].plot(ts, stddev, label="12 day")

fig.tight_layout()
plt.show()

#### ATR
Calculate the ATR. Each entry is assumed to be a day.

In [ ]:
def true_range(high, low, yest_closing):
    # ATR by J. Welles Wilder Jr.

    # Check argument validity
    if low > high:
        raise Exception
    if yest_closing == None:
        return high-low
    return max(high-low, abs(high-yest_closing), abs(low-yest_closing))

def atr(df, period=14):
    # For iterative speed
    close = df['close_price'].to_numpy()
    high = df['high_price'].to_numpy()
    low = df['low_price'].to_numpy()

    # Calculate all true ranges
    tr = np.empty(df.shape[0])
    tr[0] = true_range(high[0], low[0], None)
    for i in range(1, len(tr)):
        tr[i] = true_range(high[i], low[i], close[i-1])

    # The first <period>-1 ATRs should be empty
    atr = np.empty(df.size)
    for i in range(period-1):
        atr[i] = np.NaN

    # Calculate the ATR of <period>
    atr[period-1] = np.mean(tr[:period])

    # Calculate the rest of the ATRs
    for i in range(period, len(tr)):
        atr[i] = (atr[i-1]*(period-1) + tr[i]) / period

    df[f'{period:.0f}_day_atr'] = pd.Series(atr)
    return df

for i in range(len(etf_dfs)):
    etf_dfs[i] = atr(etf_dfs[i])

etf_dfs[0].head(20)

    

In [ ]:
import datetime

# Visualisation
fig, axs = plt.subplots(4, 2)

for i in range(len(etf_dfs)):
    try:
        atr = etf_dfs[i]["14_day_atr"]
        ts = etf_dfs[i]["timestamp"].map(lambda x: datetime.datetime.fromtimestamp(x/1000))
    except:
        # Less than 14 days old
        continue

    row = i % 4
    column = i // 4
    axs[row, column].set(ylabel="EMA", xlabel="Time")
    axs[row, column].set_title(etf_dfs[i].loc[0, "ticker"])
    axs[row, column].plot(ts, atr, label="12 day")

fig.tight_layout()
plt.show()

## Conclusions
The main takeaway is that the data is not very good. Despite being touted as 10 years of data, the data only stretches back as far as 2018 (at best; the worst offender, QIS, doesn't contain enough data points to caluclate a 200-day SMA). QIS and KRUZ are particularly poor.

Furthermore, several ETFs have single-digit number of transactions a day (BFIT, BSMR, QIS). These are either in unspecified multiples (hundreds?) or the ETFs are extremely illiquid.

Taking the data at face-value, the following preliminary results can be extracted:
- BSMR and QTJA (more recently) in particular have relatively low and stable volatilites (barring a single spike) and fairly smooth variations in EMA, suggesting that they may be good candidates for mean reversion strategies.
- BFIT, BOUT and SPSM have relatively higher volatilities and clear trends, suggesting that they may be suitable candidates for trend-following strategies.
- TGIF has high volatility but no clear trends
- KRUZ and QIS are too young to extract any insights

However, the recommendation is to get clearer data or avoid these ETFs for now.

References:
- https://blog.udemy.com/algorithmic-trading-finance/
- https://www.datacamp.com/tutorial/finance-python-trading
- https://blog.quantinsti.com/exploratory-data-analysis-python/
- https://www.sciencedirect.com/science/article/pii/S2772662223000528 (for inspiration)
- https://www.composer.trade/learn/examples-of-best-algorithmic-strategies
- https://www.aimspress.com/aimspress-data/dsfe/2022/2/PDF/DSFE-02-02-005.pdf